In [ ]:
__author__ = 'Brian Merino <brian.merino@noirlab.edu>'
__version__ = '04/16/2025' # yyyymmdd; version datestamp of this notebook
__keywords__ = ['Aladin','smash','des','delve']

# Generating MOCs with Aladin Lite v3

## Table of contents
* [Goals](#goals)
* [Summary](#summary)
* [Disclaimers and attribution](#disclaimer)
* [Imports and setup](#imports)
* [Start Aladin viewer](#Aladin)
* [Accessing the data](#Data)
* [Querying the Data Lab database](#Query)
* [Prepare MOCs](#prepare)
* [Plot the MOCs](#Plot)
* [Add MOCs to Aladin](#AladinMOCs)
* [Add individual sources to Aladin](#source)

<a class="anchor" id="goals"></a>
# Goals
Showcase how to query the Data Lab via the query client service and use the data to create Multi-Order Coverage maps (MOCs) that will then be displayed with the Aladin Lite viewer.

<a class="anchor" id="summary"></a>
# Summary
Aladin Lite is an interactive sky atlas that runs in your browser. Aladin can be used to explore the sky and has built-in functionality that makes it possible to overlay images onto the viewer and identify objects included in databases. This notebook will demonstrate how to create and overlay several MOCs onto Aladin, which would help identify overlapping datasets. 

The MOCs generated in this notebook will be used to display the survey footprints, making it possible to visualize any overlaps. 

Visualizing surveys like this could help identify whether a source of interest has already been observed by a survey in the Data Lab, which could be helpful in preparing a telescope proposal. 



<a class="anchor" id="disclaimer"></a>
# Disclaimer & attribution
If you use this notebook for your published science, please acknowledge the following:

* Data Lab concept paper: Fitzpatrick et al., <a href="http://dx.doi.org/10.1117/12.2057445">"The NOAO Data Laboratory: a conceptual overview"</a>, SPIE, 9149, 2014

* <a href="https://datalab.noirlab.edu/disclaimers.php">Data Lab disclaimer</a>

<a class="anchor" id="imports"></a>
# Imports and setup

This is the setup that is required to use the query client. The first thing to do is import the relevant Python modules.

In [ ]:
#plotting
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Standard lib
from getpass import getpass

# Data Lab
from dl import authClient as ac, queryClient as qc

#astropy
import astropy.units as u
from astropy.visualization.wcsaxes.frame import EllipticalFrame
from astropy.wcs import WCS
from astropy.table import QTable

#ipyaladin 
from mocpy import MOC
from ipyaladin import Aladin
from sidecar import Sidecar

%matplotlib inline

<a class="anchor" id="Aladin"></a>
# Start Aladin Viewer

Let's start by opening an Aladin Lite viewer using SideCar. Running the following cell will open a new window to the right of where the notebook's cells are shown. 

Note: You do not need to use Sidecar to establish an Aladin Lite viewer session. If you were to just run the first line of the cell block, Aladin Lite would be opened below the code block. You could still interact with the viewer and utilize all of Aladin Lite's tools, but the session would remain below this code block, meaning you would need to scroll back up to this cell everytime you wanted to visit the viewer. 

In [ ]:
aladin = Aladin()
with Sidecar(title='aladin_output'):
    display(aladin)

<a class="anchor" id="Data"></a>
# Accessing the data

To create the MOCs that will be overlayed onto Aladin, we will use the mocpy library. But first, we need some data. For this tutorial, we will take advantage of some of the many data sets that are hosted by the Astro Data Lab. Specifically, we will be working with data from three surveys: The **S**urvey of the **MA**gellanic **S**tellar **H**istory [(**SMASH**)](https://datalab.noirlab.edu/smash/smash.php), The **D**ark **E**negery **S**urvey [(**DES**)](https://datalab.noirlab.edu/des/index.php), and The **DEC**am **P**lane **S**urvey [(**DECaPS DR2**)](https://datalab.noirlab.edu/decaps/index.php).

### Authentication

Much of the functionality of Data Lab can be accessed without explicitly logging in (the service then uses an anonymous login). However, some capacities, such as saving the results of your queries to your virtual storage space, require a login (i.e., you will need a registered user account).

If you need to log in to Data Lab, un-comment the first line of code in the cell below and execute it:

In [ ]:
#token = ac.login(input("Enter user name: (+ENTER) "),getpass("Enter password: (+ENTER) "))
#print(token)
ac.whoAmI()

<a class="anchor" id="Query"></a>
# Querying the Astro Data Lab 

To access the data we will use to create the MOCs, we must draft some [ADQL](https://datalab.noirlab.edu/img/ADQL-20081030.pdf) queries. Like SQL, the general format of an ADQL query uses at least three commands: SELECT, FROM, and WHERE. SELECT identifies which columns you want to extract data from, FROM identifies which table you want to pull the columns from, and WHERE allows you to put constraints on your search (such as only select rows WHERE the magnitude is less than 20.0). 

We are essentially using the Astro Data Lab's [Web query interface](https://datalab.noirlab.edu/query.php) ([click here to view the web query interface user manual](https://datalab.noirlab.edu/docs/manual/UsingAstroDataLab/WebPortal/DataExplorer/WebQuery/WebQuery.html?highlight=query%20interface)) tool to obtain the data. One power feature of this tool is that you can go through the tap schema for every table hosted by the Data Lab. This is an important tool to become familiar with because most of the surveys that the Data Lab hosts contain multiple data releases, each of which can contain multiple tables. For instance, the SMASH survey has two data releases. We are interested in the second data release, but this release contains 11 tables. We are interested in the object table, so our FROM command should look like this: FROM smash_dr2.object.

Now that we know which table we are interested in, we must identify which columns to pull data from. Clicking on smash_dr2.object in the tap schema will open a new tab in the Query Interface tool, Column Information. Here, you will find all the columns in the table followed by a brief description and their data types. For this tutorial, we will be interested in the columns corresponding to the coordinates (ra and dec), g band magnitudes (mag_auto_g), and random_id. So our SELECT command should look like: SELECT ra, dec, mag_auto_g, random_id. 

Before continuing, it is important to note that although columns like ra and dec will appear in most tables, their names can change depending on the table you are working with. For the SMASH table we are working with here, they are listed as ra and dec, but if we were using the euclid_ero.fornax_vis table, we would need to call alpha_j2000 and delta_j2000

All the tables hosted by the Data Lab greatly vary in size, but most are very large. So, we will need to modify our SELECT command so it doesn't select every row of the table. We will use the TOP command to do this, which only returns the top x  values from a specified column (where x is a specific integer value). So our SELECT command can be rewritten as: SELECT TOP 25000 ra, dec, mag_auto_g, random_id.

Finally, we need to establish our search constraints using WHERE. To create the MOC for this table, we could select every row in the table and identify the boundaries of the table, but the smash_dr2.object table has 360,201,921 rows which would not only take a long time to download, but a table of this size would crash the Jupyter Notebook and Aladin Lite. To avoid this, we are going to utilize the random_id column. As the name implies, this column contains a unique random ID for each row whose values are floats in the range 0.0 and 100.0. Because the table is so large, we only need to establish a small window, say between 25.0 and 25.01. To use the WHERE command, we need to provide a column that the constraint will be applied to (random_id) and a constraint. In this case, our constraint is to only select rows whose random_id values are BETWEEN 25.0 and 25.01. So our WHERE command will look like: WHERE random_id BETWEEN 25.0 AND 25.01.

Putting all of this together leaves us with the following query:

SELECT TOP 25000 ra, dec, gmag, random_id FROM smash_dr2.object WHERE random_id BETWEEN 25.0 AND 25.01

Now that we have an ADQL query prepared for SMASH, we will need to repeat the process to query the DECaPS and DES surveys. If you find yourself getting stuck with SQL/ADQL, you can visit the following link to learn about common [SQL Gotchas](https://datalab.noirlab.edu/docs/manual/UsingAstroDataLab/SQLGotchas/index.html?highlight=sql).

In [ ]:
query = 'SELECT TOP 25000 ra, dec, mag_auto_g, random_id FROM des_dr2.main WHERE random_id BETWEEN 25.0 AND 25.01'
des = qc.query(sql = query, fmt = 'pandas')

# Let's add two additional columns to the surveys
# The first assigns a unique index to each data point.
# The second attaches the survey name to each row
des['index'] = list(range(0,len(des)))
des['survey'] = 'des_dr2'

print('des_dr2 catalog')
print (des)

In [ ]:
query2 = 'SELECT TOP 25000 ra, dec, gmag, random_id FROM smash_dr2.object WHERE random_id BETWEEN 25.0 AND 25.01'
smash = qc.query(sql = query2, fmt = 'pandas')

smash['index'] = list(range(0,len(smash)))
smash['survey'] = 'smash_dr2'

print('smash_dr2')
print (smash)

In [ ]:
query3 = 'SELECT TOP 25000 ra, dec, mean_r, random_id FROM decaps_dr2.object WHERE random_id BETWEEN 25.0 AND 25.1'
decaps = qc.query(sql = query3, fmt = 'pandas')

decaps['index'] = list(range(0,len(decaps)))
decaps['survey'] = 'decaps_dr2'

print('decaps_dr2')
print (decaps)

<a class="anchor" id="prepare"></a>
# Prepare MOCs

Now that all three datasets have been downloaded, we can now create Multi-Order Coverage maps (MOCs) for each of them using the mocpy library. 

In [ ]:
def prepare(columns, dataframe, max_norder):
    '''
    This function will perform two tasks:
    (1) Read in columns of a dataframe and output a dictionary that is compatible with Aladin Lite

    (2) Use the ra and dec info from the dictionary to create a MOC.
    '''
    # Define a dictionary
    col_dict = {}
    
    # Read each item in the provided list and extract the values from the dataframe. Assign values to the dictionary.
    for c in columns:
        col_dict[c] = dataframe[c].values
        
    # Now we are going to establish our MOC.  
    # Provide MOC.from_longlat() with ra and dec in degrees
    # max_norder:The depth of the smallest HEALPix cells contained in the MOC. Min = 1 | Max = 10
    moc = MOC.from_lonlat(
              col_dict['ra'].transpose() * u.deg,
              col_dict['dec'].transpose() * u.deg,
              max_norder=max_norder)

    return col_dict, moc

Before calling the prepare() function, we will need to create a list for each survey that contains the column names that we want to pass to Aladin. 

In [ ]:
des_list = ['ra','dec','index','mag_auto_g','survey']
des_dict, des_dr2_moc = prepare(des_list, des, max_norder=5)

smash_list = ['ra','dec','index','gmag','survey']
smash_dict, smash_dr2_moc = prepare(smash_list, smash, max_norder=5)

decaps_list = ['ra','dec','index','mean_r','survey']
decaps_dict, decaps_dr2_moc = prepare(decaps_list, decaps, max_norder=5)

<a class="anchor" id="Plot"></a>
# Plot the MOCs

Before displaying the MOCs on Aladin, lets see what they look like using Matplotlib.

In [ ]:
def plot(moc, color, legend, title=""):
    fig = plt.figure(figsize=(15, 10))

    for c,m in enumerate(moc):
        wcs = WCS(naxis=2)
        wcs.wcs.ctype = ["GLON-AIT", "GLAT-AIT"] #Hammer-Aitoff projection
        wcs.wcs.crval = [300.0, 0.0]             #Specify the value of the reference pixel
        wcs.wcs.cdelt = [-0.675, 0.675]
        wcs.wcs.crpix = [240.5, 120.5]
    
        if c == 0:
            ax = fig.add_subplot(1, 1, 1, projection=wcs, frame_class=EllipticalFrame)
            patches = []
    
        m.fill(
            ax=ax,
            wcs=wcs,
            edgecolor=color[c],
            facecolor=color[c],
            linewidth=1.0,
            fill=True,
            alpha=0.5,
        )
        m.border(ax=ax, wcs=wcs, color="black", alpha=1)
    
        plt.xlabel("ra")
        plt.ylabel("dec")
        
        if title:
            plt.title(title)
        plt.grid(color="black", linestyle="dotted")
        
        patches.append(mpatches.Patch(color=color[c], label=legend[c]))
        plt.legend(handles=patches)
        
    plt.show()
    plt.close()

In [ ]:
moc = [des_dr2_moc, smash_dr2_moc, decaps_dr2_moc]
legend = ['des_dr2_moc', 'smash_dr2_moc', 'decaps_dr2_moc']
colors = ['cyan', 'red', 'blue', 'yellow', 'purple']
plot(moc,title='',color=colors,legend=legend)

<a class="anchor" id="AladinMOCs"></a>
# Add the MOC to Aladin

Now that we have seen the survey coverage using Matplotlib, let's use ipyaladin's add_moc() function to overlay the MOCs onto Aladin.

In [ ]:
aladin.add_moc(des_dr2_moc, color='red',   name='des_dr2_moc', opacity=0.4)
aladin.add_moc(smash_dr2_moc, color='blue',  name='smash_dr2_moc', opacity=0.4)
aladin.add_moc(decaps_dr2_moc, color='yellow', name='decaps_dr2_moc', opacity=0.4)

<a class="anchor" id="source"></a>
# Display individual sources on Aladin

Now, let's plot the individual sources onto Aladin. Currently, Aladin Lite only works reliably with QTables, so we are going to reformat our pandas data frames into something the Aladin viewer will accept. 

In [ ]:
#des_dr2
des_table    = QTable([des_dict['ra'],des_dict['dec'],des_dict['index'],\
                      des_dict['mag_auto_g'],des_dict['survey']],\
                      names=["ra","dec","index","mag_auto_g","survey"])

#smash_dr2
smash_table  = QTable([smash_dict['ra'],smash_dict['dec'],smash_dict['index'],\
                      smash_dict['gmag'],smash_dict['survey']],\
                      names=["ra","dec","index","gmag","survey"])

#decaps_dr2
decaps_table = QTable([decaps_dict['ra'],decaps_dict['dec'],decaps_dict['index'],\
                      decaps_dict['mean_r'],decaps_dict['survey']],\
                      names=["ra","dec","index","mean_r","survey"])

With our datasets reformatted, we can add now display on the Aladin viewer using the add_table() function.  

In [ ]:
aladin.add_table(des_table,name='des_dr2')
aladin.add_table(smash_table,name='smash_dr2')
aladin.add_table(decaps_table,name='decaps_dr2')

By now, your Aladin viewer maybe hard to read since there are many MOCs and individual data points plotted ontop of each other. You can remove some of these objects by clicking the 'Overlays menu' in your Aladin viewer. A dropdown menu will appear displaying the names of all the MOCs and data tables currently being displayed. From there, you can hover your mouse over any item and either click on the 'eye' icon to hide the object, or the 'trash' icon to fully remove it from the viewer. 

![Pointing out the location of the hide and remove icons in Aladin Lite.](Remove_layer.png "Remove layer.")